In [1]:
from models import TransformerNet
import torch
import argparse
import os
import coremltools as ct
from utils import *

scikit-learn version 1.3.0 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
Torch version 2.0.0 has not been tested with coremltools. You may run into unexpected errors. Torch 1.13.1 is the most recent version that has been tested.


In [2]:
transform = style_transform()

# device = torch.device("mps" if torch.cuda.is_available() else "cpu")

device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"


path = "pre-trained/starry_night_10000.pth"

transformer = TransformerNet().to(device)
transformer.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
transformer.eval()
print(device)

# path = "starry_night_10000.pth"
name = path.split("/")
print(name)
name = name[-1]
name = name.split("_")[0] + "_" + name.split("_")[1]
print(name)

mps
['pre-trained', 'starry_night_10000.pth']
starry_night


In [3]:
# path = "pre-trained/starry_night_10000.pth"
# parts = path.split("/")
# print(parts)
# filename = parts[-1]  # 获取文件名部分
# print(filename)
# name_parts = filename.split("_")  # 以下划线分割文件名
# desired_name = "_".join(name_parts[:-1])  # 组合前面的部分，不包括最后一个下划线后的部分

# print(desired_name)

In [3]:
example_input = torch.randn(1,3,960,640).to(device)
traced_model = torch.jit.trace(transformer, example_input)

mlmodel = ct.convert(traced_model, inputs=[ct.ImageType(name="input", shape=example_input.shape,bias=[-0.485/0.229,-0.456/0.224,-0.406/0.225],scale=1.0/255.0/0.226)])

Running MIL Common passes:   0%|          | 0/40 [00:00<?, ? passes/s]/Users/jason/miniconda3/envs/torch/lib/python3.9/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:135: UserWarning: Output, '294', of the source model, has been renamed to 'var_294' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 244/244 [00:00<00:00, 292.11 ops/s]


In [4]:
spec = mlmodel.get_spec()
builder = ct.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.spec.description

input {
  name: "input"
  type {
    imageType {
      width: 640
      height: 960
      colorSpace: RGB
    }
  }
}
output {
  name: "var_294"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.0.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "6.2"
  }
}

In [47]:
# builder.spec.description
# print(type(builder.spec.description))
print(builder.spec.description.output)
    
output_name = spec.description.output[0].name

print("Output Name:", output_name)

[name: "var_294"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]
Output Name: var_294


In [20]:
builder.add_split(name="split", input_name="var_294", output_names=["split_1","split_2","split_3"])
builder.add_activation(name="activation_1",non_linearity="LINEAR",input_name="split_1",output_name="activation_out_1",params=[255*0.226,0.485*255])
builder.add_activation(name="activation_2",non_linearity="LINEAR",input_name="split_2",output_name="activation_out_2",params=[255*0.226,0.456*255])
builder.add_activation(name="activation_3",non_linearity="LINEAR",input_name="split_3",output_name="activation_out_3",params=[255*0.226,0.406*255])
builder.add_stack(name="stack", input_names=["activation_out_1","activation_out_2","activation_out_3"], output_name="stack_out", axis=0)
builder.add_squeeze(name="squeeze", input_name="stack_out", output_name="squeeze_out", axes = None, squeeze_all = True)

name: "squeeze"
input: "stack_out"
output: "squeeze_out"
squeeze {
  squeezeAll: true
}

In [21]:
from coremltools.proto import FeatureTypes_pb2 as ft
from coremltools.models.utils import save_spec

In [22]:
builder.spec.description.output.pop()
builder.spec.description.output.add()
output = builder.spec.description.output[0]
output.name = "squeeze_out"

output.type.imageType.colorSpace = ft.ImageFeatureType.ColorSpace.Value('RGB')
output.type.imageType.width = 640 
output.type.imageType.height = 960

save_spec(builder.spec, 'fast-style-transfer-starry_night.mlmodel')
print("done")

done


In [ ]:
from coremltools.models.neural_network import quantization_utils
